In [19]:
import pandas as pd
import os

from typing import Dict

# Formatting Original Data

In [33]:
src_dir = os.getcwd()
data_dir = os.path.join(src_dir, "Data")
os.makedirs(data_dir, exist_ok=True)  # Make directory if it doesn't exist

## nba-players-stats/Seasons_Stats.csv

In [34]:
stat_50_17_file = os.path.join(src_dir, "nba-players-stats/Seasons_Stats.csv")
stat50_17 = pd.read_csv(stat_50_17_file, index_col=0)

In [35]:
def format_stats_50_17(stat50_17: pd.DataFrame, type_conv: Dict) -> pd.DataFrame:
    """
    Formats 1950-2017 stats file. Note that this function modifies stat50_17 to save memory.
    In:
        stat50_17: DataFrame from 1950-2017 stats file; modified in-place to save memory.
        type_conv: Dict containing column names to convert to int.
    Return:
        Formatted 1991-2017 stats DataFrame.
    """
    # Drop rows prior to 1990 (our salary data is from 1991 onwards)
    stat50_17.drop(stat50_17[stat50_17["Year"] < 1990].index, inplace=True)
    # Drop NA columns
    stat50_17.dropna(axis=1, how='all', inplace=True)
    # Drop NA rows
    stat50_17.dropna(inplace=True)  # Defaults: axis=0, how='any'
    # For players who had multiple teams for a year, keep the TOTAL row
    stat50_17.drop_duplicates(subset=["Year", "Player"], inplace=True)
    # Convert certain cols to int; deep copy here
    stat90_17 = stat50_17.astype(type_conv)
    # Remove asterisks from HOF players
    stat90_17["Player"] = stat90_17["Player"].str.rstrip('*')
    # Reset index
    stat90_17.reset_index(drop=True, inplace=True) # Drop the old index (rather than adding it as a col)
    return stat90_17

In [39]:
cols_to_int = ['Year', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
              'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'] 
type_conv = dict.fromkeys(cols_to_int, int)
stat90_17 = format_stats_50_17(stat50_17, type_conv)
# Rename Year col to Season End for clarity
stat90_17.rename(columns={"Year": "Season End"}, inplace=True) 
# Show first and last 5 rows and columns
# iloc[row_idx_list, col_idx_list]
print(stat90_17.iloc[list(range(5)) + list(range(-5, 0)), 
                   list(range(5)) + list(range(-5, 0))])

       Season End          Player Pos  Age   Tm  STL  BLK  TOV   PF   PTS
0            1990      Mark Acres   C   27  ORL   36   25   70  248   362
1            1990   Michael Adams  PG   27  DEN  121    3  141  133  1221
2            1990    Mark Aguirre  SF   30  DET   34   19  121  201  1099
3            1990     Danny Ainge  PG   30  SAC  113   18  185  238  1342
4            1990     Mark Alarie  PF   26  WSB   60   39  101  219   860
10208        2017  Thaddeus Young  PF   28  IND  114   30   96  135   814
10209        2017     Cody Zeller  PF   24  CHO   62   58   65  189   639
10210        2017    Tyler Zeller   C   27  BOS    7   21   20   61   178
10211        2017     Paul Zipser  SF   22  CHI   15   16   40   78   240
10212        2017     Ivica Zubac   C   19  LAL   14   33   30   66   284


In [40]:
out_stats_name = "nba_stats_1990_2017.csv"
out_stats_path = os.path.join(data_dir, out_stats_name)

In [41]:
if not os.path.isfile(out_stats_path): stat90_17.to_csv(out_stats_path, index=False)

## Salaries Per Season (1990-2017)

In [59]:
sal91_18_file = os.path.join(src_dir, "nba-player-salary-19902017/Player - Salaries per Year (1990 - 2017).csv")
sal91_18 = pd.read_csv(sal91_18_file, thousands=",")

In [73]:
def format_sal(sal91_18: pd.DataFrame) -> pd.DataFrame:
    """
    Formats 1991-2018 salaries file. Note that this function modifies sal91_18 to save memory.
    In:
        sal91_18: DataFrame from 1991-2018 salaries file; modified in-place to save memory.
    Return:
        Formatted salaries DataFrame.
    """
    # Remove "Register Value, Team, Full Team Name" columns
    if "Register Value" in sal91_18.columns: sal91_18.pop("Register Value")
    if "Team" in sal91_18.columns: sal91_18.pop("Team")
    if "Full Team Name" in sal91_18.columns: sal91_18.pop("Full Team Name")
    # Rename Salary in $ to Salary
    sal91_18.rename(columns={" Salary in $ ": "Salary"}, inplace=True) 
    # Convert Salary to int
    sal91_18['Salary'] = sal91_18['Salary'].str.replace(r'[\$,]', '').astype(int)
    return sal91_18

In [74]:
sal91_18 = format_sal(sal91_18)
print(sal91_18.head())
print(sal91_18.tail())

ValueError: invalid literal for int() with base 10: ' 1750000.00 '

In [62]:
sal91_18.columns

Index(['Player Name', ' Salary in $ ', 'Season Start', 'Season End'], dtype='object')

In [45]:
out_sal_name = "nba_salaries_1991_2018.csv"
out_sal_path = os.path.join(data_dir, out_sal_name)

In [46]:
if not os.path.isfile(out_sal_path): sal91_18.to_csv(out_sal_path, index=False)

## Matching Salary Data to Player Data by Year

In [50]:
sal91_18_form_file = os.path.join(data_dir, "nba_salaries_1991_2018.csv")
sal91_18_form = pd.read_csv(sal91_18_form_file)
stat90_17_form_file = os.path.join(data_dir, "nba_stats_1990_2017.csv")
stat90_17_form = pd.read_csv(stat90_17_form_file)